In [1]:
import sys
import math

from src.raytracer.raytracer import point3, vec3, color, unit
from src.raytracer.raytracer import unit, dot
from src.raytracer.raytracer import random_in_hemisphere, random_unit_vector
from src.raytracer.ray import ray
from src.raytracer import hittables, materials, textures
from src.raytracer import camera, writeimg, rweekend
from datetime import datetime
import pytz

In [2]:
# used to get a color for a ray
def ray_color(r, world, depth):
    rec = hittables.hit_record()
    if depth <= 0:
        return color(0, 0, 0)
    
    hit_out = world.hit(r, 0.001, rweekend.infinity, rec)
    if hit_out[0]:
        rec = hit_out[1]
        
        scattered = ray()
        attenuation = vec3()
        out = rec.material.scatter(r, rec, attenuation, scattered)
        if out[0]:
            scattered = out[1]
            attenuation = out[2]
            return (attenuation * ray_color(scattered, world, depth-1))
        return color(0, 0, 0)

    # make background gradient
    unit_direction = unit(r.direction)
    t = 0.5 * (unit_direction.y + 1.0)
    return (1.0 - t) * color(1.0, 1.0, 1.0) \
            + t * color(0.5, 0.7, 1.0)

In [3]:
# setup code for scene with a bunch of random spheres
def random_scene():
    world = hittables.hittable_list()
    
    ground_material = materials.lambertian(color(0.5, 0.5, 0.5))
    world.add(hittables.sphere(point3(0.0, -1000, 0.0), 1000.0, ground_material))
    
    for a in range(-2, 2):
        for b in range(-2, 2):
            choose_mat = rweekend.random_double()
            center = point3(a + 0.9 * rweekend.random_double(), 0.2, b + 0.9 * rweekend.random_double())
            
            if ((center - point3(4.0, 0.2, 0.0)).len > 0.9):
                if (choose_mat < 0.8):
                    # diffuse sphere
                    albedo = color.random() * color.random()
                    sphere_material = materials.lambertian(albedo)
                    center2 = center + vec3(0.0, rweekend.random_double(0.0, 0.5), 0.0) # add moving sphere
                    world.add(hittables.moving_sphere(center, center2, 0.0, 1.0, 0.2, sphere_material))
                elif (choose_mat < 0.95):
                    # metal sphere
                    albedo = color.random(0.5, 1.0)
                    fuzz = rweekend.random_double(0.0, 0.5)
                    sphere_material = materials.metal(albedo, fuzz)
                    world.add(hittables.sphere(center, 0.2, sphere_material))
                else:
                    # glass sphere
                    sphere_material = materials.dielectric(1.5)
                    world.add(hittables.sphere(center, 0.2, sphere_material))

    material1 = materials.dielectric(1.5)
    world.add(hittables.sphere(point3(0.0, 1.0, 0.0), 1.0, material1))
    
    material2 = materials.lambertian(color(0.4, 0.2, 0.1))
    world.add(hittables.sphere(point3(-4.0, 1.0, 0.0), 1.0, material2))
    
    material3 = materials.metal(color(0.7, 0.6, 0.5), 0.0)
    world.add(hittables.sphere(point3(4.0, 1.0, 0.0), 1.0, material3))
    
    return world


In [4]:
# scene setup for two large spheres
def two_spheres():
    objects = hittables.hittable_list()
    
    checker = textures.checker_texture(color(0.2, 0.3, 0.1), color(0.9, 0.9, 0.9))
    objects.add(hittables.sphere(point3(0.0, -10, 0.0), 10.0, materials.lambertian(checker)))
    objects.add(hittables.sphere(point3(0.0, 10, 0.0), 10.0, materials.lambertian(checker)))
    
    return objects
    

In [5]:
def two_perlin_spheres():
    objects = hittables.hittable_list()
    
    perlin_text_basic = textures.noise_texture()
    objects.add(hittables.sphere(point3(0.0, -1000.0, 0.0), 1000.0, materials.lambertian(perlin_text_basic)))
    objects.add(hittables.sphere(point3(0.0, 2.0, 0.0), 2.0, materials.lambertian(perlin_text_basic)))
    
    return objects


In [6]:
##

In [7]:
# choose scene
scene_number = 3

In [8]:
# scene defaults
vfov = 40.0
aperture = 0.0

In [9]:
# scene vars per scene

# random small spheres and 3 big ones
if scene_number == 1:
    world = random_scene()
    lookfrom = point3(13, 2, 3)
    lookat = point3(0, 0, 0)
    vfov = 20.0
    aperture = 0.1
    
# 2 large checker spheres
elif scene_number == 2:
    world = two_spheres()
    lookfrom = point3(13, 2, 3)
    lookat = point3(0, 0, 0)
    vfov = 20.0
    
# 1 small and 1 large perlin sphere
elif scene_number == 3:
    world = two_perlin_spheres()
    lookfrom = point3(13, 2, 3)
    lookat = point3(0, 0, 0)
    vfov = 20.0

In [10]:
# common camera params
vup = vec3(0,1,0)
dist_to_focus = 10.0
aspect_ratio = 16.0 / 9.0
cam = camera.camera(lookfrom, lookat, vup, 20.0, aspect_ratio, 
                    aperture, dist_to_focus, 0.0, 1.0)

In [11]:
# image params
image_width = 400
image_height = math.floor(image_width / aspect_ratio)
samples_per_pixel = 2 
max_depth = 5

In [12]:
##

In [13]:
# print start time
tz_CH = pytz.timezone('America/Chicago') 
print("start time: ", datetime.now(tz_CH).strftime("%H:%M:%S"))

start time:  21:50:12


In [14]:
# render image
outimg = writeimg.writeppm(image_width, image_height,
                           'outfile.ppm', 'P3', 255)
outimg.write_head()
for j in range(image_height-1, -1, -1):
    sys.stdout.write("\r%d%%" % j)
    sys.stdout.flush()
    for i in range(0, image_width):
        pixel_color = color(0, 0, 0)
        for s in range(0, samples_per_pixel):
            u = float(i + rweekend.random_double())/(image_width - 1)
            v = float(j + rweekend.random_double())/(image_height - 1)
            r = cam.get_ray(u, v)
            pixel_color += ray_color(r, world, max_depth)
            #print("pixel_color", pixel_color)
        outimg.write_color(pixel_color, samples_per_pixel)
sys.stdout.write("done")

0%%%done

In [15]:
# check if valid file vars
outimg.check_valid()

[True, 'params OK']

In [16]:
# write ppm file
outimg.write_color_file()

In [17]:
# print start time
print("end time: ", datetime.now(tz_CH).strftime("%H:%M:%S"))

end time:  21:51:25
